In [1]:
import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr
from qcodes.instrument.parameter import Parameter
from qc_measure_20240712 import *
print_versions()

qcodes: 0.46.0
qcodes_contrib_drivers: 0.22.0
plottr: 0.13.0
python: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
matplotlib: 3.8.4
numpy: 1.26.4
scipy: 1.14.0
pandas: 2.2.2


In [2]:
# Create a new db file name for each cooldown.
db_path = 'test 20240707-XL105 20221222-InAs2DEGJJ15 username.db'

qc.logger.start_all_logging()
initialise_or_create_database_at(db_path)

station = qc.Station()

# Open Plottr
# jobs = bg.BackgroundJobManager()
# jobs.new(inspectr.main, db_path)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Measurement\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Measurement\.qcodes\logs\240713-6048-qcodes.log


In [3]:
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
# qdac.close()
qdac = QDAC2.QDac2('QDAC',  address= 'COM4')

from qcodes.instrument_drivers.Keysight.Keysight_34461A_submodules import Keysight_34461A
# dmm.close()
dmm =  Keysight_34461A('Keysight_34461A', 'USB0::0x2A8D::0x1301::MY60060848::INSTR')

Connected to: QDevil QDAC-II (serial:78, firmware:7-0.17.5) in 0.14s
Connected to: Keysight Technologies 34461A (serial:MY60060848, firmware:A.03.03-03.15-03.03-00.52-04-03) in 0.16s


In [4]:
def get_V_measure():
    return dmm.volt()

V_measure = Parameter('V_measure', unit='V', label='Vmeas', 
                                 get_cmd=get_V_measure, set_cmd=None)


Dummy_x = DummyParameter(name='dummy_x',unit='')

Dummy_y = DummyParameter(name='dummy_y',unit='')

Vg = qdac.ch01.dc_constant_V

In [5]:
Ibias_fast = DummyParameter(name='Ibias_fast', unit='uA')

# Test 

In [16]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in self.tqdm_x(list_x):
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            datasaver.add_result(*rslt)
    
            if self.is_fwd_now or self.scan_dim==1:
                self.qclient.add_data([i[1] for i in rslt])
                self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=True)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code')
qm.scan2d(bwd=False)

Test: original code
Starting experimental run with id: 137. 


Total: 6.090047121047974 s. Each line: 5.08890700340271 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.137


In [18]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in self.tqdm_x(list_x):
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            datasaver.add_result(*rslt)
    
            if self.is_fwd_now or self.scan_dim==1:
                self.qclient.add_data([i[1] for i in rslt])
                self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=False)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code without parallel')
qm.scan2d(bwd=False)

Test: original code without parallel
Starting experimental run with id: 139. 


Total: 2.482361316680908 s. Each line: 1.4816839694976807 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.139


In [24]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in self.tqdm_x(list_x):
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            datasaver.add_result(*rslt)
    
            # if self.is_fwd_now or self.scan_dim==1:
            #     self.qclient.add_data([i[1] for i in rslt])
            #     self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=False)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code without parallel, without qclient')
qm.scan2d(bwd=False)

Test: original code without parallel, without qclient
Starting experimental run with id: 145. 


Total: 2.4096851348876953 s. Each line: 1.4081535339355469 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.145


In [25]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in self.tqdm_x(list_x):
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            # datasaver.add_result(*rslt)
    
            # if self.is_fwd_now or self.scan_dim==1:
            #     self.qclient.add_data([i[1] for i in rslt])
            #     self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=False)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code without parallel, without qclient, without datasaver.add_result()')
qm.scan2d(bwd=False)

Test: original code without parallel, without qclient, without datasaver.add_result()
Starting experimental run with id: 146. 


Total: 1.53621506690979 s. Each line: 0.535008430480957 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.146


In [26]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in list_x:
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            # datasaver.add_result(*rslt)
    
            # if self.is_fwd_now or self.scan_dim==1:
            #     self.qclient.add_data([i[1] for i in rslt])
            #     self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=False)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code without parallel, without qclient, without datasaver.add_result(), without tqdm')
qm.scan2d(bwd=False)

Test: original code without parallel, without qclient, without datasaver.add_result(), without tqdm
Starting experimental run with id: 147. 
Total: 1.455249309539795 s. Each line: 0.4547867774963379 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.147


In [27]:
class QMeasure_test(QMeasure):

    def to_time_str(self, t0,t1,t2,t3):
        return f'Total: {t3-t0} s. Each line: {t2-t1} s.'
    
    def _scan1d(self, datasaver, list_x, y):
        para_x = self.scan_para['x']['parameter']
        delay_x = self.scan_para['x']['delay']
        para_y = self.scan_para['y']['parameter']
        
        for x in list_x:
            para_x(x)
            time.sleep(delay_x)
            vals = self._take_data()
            rslt = [(para_x, x), (para_y, y)] + vals
            # datasaver.add_result(*rslt)
    
            # if self.is_fwd_now or self.scan_dim==1:
            #     self.qclient.add_data([i[1] for i in rslt])
            #     self.qclient.update_plot()


qm = QMeasure_test(exp_name='Ibias',sample_name='left-12-13-a',export_dat=True, mute_qclient=False)
qm.set_parameters_to_aquire(para_list=[Ibias_fast], parallel=False)
qm.set_parameters_in_measurement_name(para_list=[V_measure, dmm.volt])

qm.set_scan_parameter('x', para=Dummy_x, start=0, stop=1, points=10000, delay=0.001)
# qm.set_scan_parameter('y', para=None, start=None, stop=None, points=None, delay=0)

print('Test: original code without parallel, without qclient, without datasaver.add_result(), without tqdm, sleep 0.001')
qm.scan2d(bwd=False)

Test: original code without parallel, without qclient, without datasaver.add_result(), without tqdm, sleep 0.001
Starting experimental run with id: 148. 
Total: 19.591214179992676 s. Each line: 18.590445280075073 s.
DAT and SET file saved: DAT\Ibias.left-12-13-a.148


In [28]:
%timeit Ibias_fast(0)

23.7 μs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
%timeit Ibias_fast()

18.9 μs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
%timeit qm._take_data()

21.6 μs ± 498 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
